# Split into 10 seconds

In [ ]:
from tqdm import tqdm
import re

def add_offset_for_long_audios(csv_path, max_audio_duration):
    with open(csv_path, 'r') as f:
        # if files longer than 10 seconds, then split them into 10 seconds
        lines = f.readlines()
        print(len(lines))
        index = 0
        index_list = []
        lines_to_add = []
        line_count = 0
        lines_to_add.append('ID,utt_id,wav,wav_format,text,duration,offset,accent\n')
        make_state_level = True
        for line in tqdm(lines[1:]):
            # print("line: ", line)
            line_count += 1
            line = line.strip()
            line = line.split(',')
            wav_file = line[2]
            duration = float(line[5])
            if make_state_level==True:
                if line[6][0] == '1':    # check first number of line[6]
                    line[6] = '1'
                elif line[6][0] == '2':
                    line[6] = '2'
                elif line[6][0] == '3':
                    line[6] = '3'
                elif line[6][0] == '4':
                    line[6] = '4'
                elif line[6][0] == '5':
                    line[6] = '5'
                elif line[6][0] == '6':
                    line[6] = '6'
                elif line[6][0] == '7':
                    line[6] = '7'
                elif line[6][0] == '8':
                    line[6] = '8'                
            # print(f"Duration: {duration} for {wav_file}")
            if duration > max_audio_duration:
                split_files = []
                for i in range(0, int(duration), max_audio_duration):
                    start = i   # = offset_time
                    end = i + max_audio_duration
                    # split_file = wav_file.split('.wav')[0] + f'_{start}_{end}.wav'
                    offset_time = start
                    ##
                    if duration-start>10:
                        # print("duration: 10")
                        duration_chunk = max_audio_duration
                    else:
                        # print(f"duration: {duration-start}")
                        duration_chunk = np.round(duration-start, 3)
                    ##
                    # if longer than max_audio then copy name and add offset to offset_column
                    offset_time = start
                    lines_to_add.append(f'{line[0]}, {line[1]}, {line[2]}, {line[3]}, {line[4]}, {duration_chunk}, {offset_time}, {line[6]}\n')
                # handle the last part of the audio file
                if duration % max_audio_duration != 0 and duration % max_audio_duration >= 0.5 and duration % max_audio_duration < 1:   # only add for durations like 10.6 otherwise it is covered by above. Everything where int(duration) % 10 >= 1 is covered by above. So this is only for 0.5 seconds segments after 10, 20, 30, ...
                    start = int(duration) - (int(duration) % max_audio_duration)
                    if duration - start >= 0.5:
                        duration_chunk = np.round(duration-start, 3)
                        offset_time = start
                        lines_to_add.append(f'{line[0]}, {line[1]}, {line[2]}, {line[3]}, {line[4]}, {duration_chunk}, {offset_time}, {line[6]}\n')
                    else:
                        # print("Last part of the audio file is less than 0.5 seconds")
                        continue
            else:
                # leave everything as it is and add an offset of 0
                lines_to_add.append(f'{line[0]}, {line[1]}, {line[2]}, {line[3]}, {line[4]}, {line[5]}, 0, {line[6]}\n')
                
    # Separate the header from the data
    header = lines_to_add[0]
    data_lines = lines_to_add[1:]
    
    # Reset the ID column, starting from 0
    new_data_lines = []
    for i, line in enumerate(data_lines):
        parts = line.split(',')
        new_id = i  # IDs start from 0
        new_line = f'{new_id},{",".join(parts[1:])}'
        new_data_lines.append(new_line)
        
    # Combine header with new data lines
    updated_lines_to_add = [header] + new_data_lines
    
    # remove double blank spaces
    updated_lines_to_add = [re.sub(' +', '', line) for line in updated_lines_to_add]  # plus means one or more spaces

    return updated_lines_to_add

# test the function
split_duration = 10
updated_lines_to_add_train = add_offset_for_long_audios('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at/train.csv', split_duration)
updated_lines_to_add_dev = add_offset_for_long_audios('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at/dev.csv', split_duration)
updated_lines_to_add_test = add_offset_for_long_audios('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at/test.csv', split_duration)

# Write the updated data to a new file
with open('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at/train_offset2.csv', 'w') as f:
    for line in updated_lines_to_add_train:
        f.write(line)

with open('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at/dev_offset2.csv', 'w') as f:
    for line in updated_lines_to_add_dev:
        f.write(line)
        
with open('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at/test_offset2.csv', 'w') as f:
    for line in updated_lines_to_add_test:
        f.write(line)

# create devolpment set
updated_lines_to_add_train_dev = updated_lines_to_add_train[:1000]
updated_lines_to_add_dev_dev = updated_lines_to_add_dev[:100]
updated_lines_to_add_test_dev = updated_lines_to_add_test[:100]

# Write the updated data to a new file
with open('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_dev/train_offset2.csv', 'w') as f:
    for line in updated_lines_to_add_train_dev:
        f.write(line)

with open('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_dev/dev_offset2.csv', 'w') as f:
    for line in updated_lines_to_add_dev_dev:
        f.write(line)

with open('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_dev/test_offset2.csv', 'w') as f:
    for line in updated_lines_to_add_test_dev:
        f.write(line)


In [1]:
from tqdm import tqdm
import re

def minimum_duration(csv_path, min_audio_duration):
    with open(csv_path, 'r') as f:
        # if files longer than 10 seconds, then split them into 10 seconds
        lines = f.readlines()
        # print(len(lines))
        lines_to_add = []
        lines_to_add.append('ID,utt_id,wav,wav_format,text,duration,offset,accent\n')
        for line in tqdm(lines[1:]):
            line = line.strip()
            line = line.split(',')
            if float(line[5]) >= float(min_audio_duration):
                lines_to_add.append(f'{line[0]},{line[1]},{line[2]},{line[3]},{line[4]},{line[5]},{line[6]},{line[7]}\n')
            else:
                # nothing is added
                continue
    return lines_to_add

# test the function
min_duration = 2
train_csv = '/home/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_states/train_all.csv'
updated_lines_to_add_train = minimum_duration(train_csv, min_duration)
dev_csv = '/home/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_states/dev_all.csv'
updated_lines_to_add_dev = minimum_duration(dev_csv, min_duration)
test_csv = '/home/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_states/test_all.csv'
updated_lines_to_add_test = minimum_duration(test_csv, min_duration)

# Write the updated data to a new file
with open('/home/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_states/train_mindur2.csv', 'w') as f:
    for line in updated_lines_to_add_train:
        f.write(line)

with open('/home/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_states/dev_mindur2.csv', 'w') as f:
    for line in updated_lines_to_add_dev:
        f.write(line)
        
with open('/home/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_states/test_mindur2.csv', 'w') as f:
    for line in updated_lines_to_add_test:
        f.write(line)

100%|██████████| 7381/7381 [00:00<00:00, 3005062.88it/s]


# Read classification report

In [68]:
import pandas as pd
import numpy as np

with open('/nas/projects/vokquant/accent-recog-slt2022/CommonAccent/results/ECAPA-TDNN/AT/spkrec-ecapa-voxceleb/10000/classification_report_test.txt', 'r') as f:
    lines = f.readlines()
    accent_dict = {}
    for line in lines[2:]:
        if line.strip() == "":
            break
        split_line = line.split()
        city_code = split_line[0]
        precision = split_line[1]
        recall = split_line[2]
        f1_score = split_line[3]
        support = split_line[4]
        accent_dict[city_code] = [precision, recall, f1_score, support]
        
accent_df = pd.DataFrame.from_dict(accent_dict, orient='index', columns=['precision', 'recall', 'f1_score', 'support'])
# name index to city_code
accent_df.index.name = 'city_code'
accent_df
# convert precision, recall, f1_score, and support to float
accent_df.loc[:, 'precision'] = accent_df['precision'].astype(float)
accent_df.loc[:, 'recall'] = accent_df['recall'].astype(float)
accent_df.loc[:, 'f1_score'] = accent_df['f1_score'].astype(float)
accent_df.loc[:, 'support'] = accent_df['support'].astype(float)
# find all city codes starting with a 1
region_1 = accent_df[accent_df.index.str.startswith('1')]
rounded_means = region_1.mean().round(3)
# round to three decimals
print(rounded_means)

precision         0.615
recall         0.596667
f1_score       0.573333
support      104.166667
dtype: object
